## 梅爾倒頻譜
在語音識別（Speech Recognition）和話者識別（Speaker Recognition）方面，最常用到的語音特徵就是梅爾倒譜係數（Mel-scale Frequency Cepstral Coefficients，簡稱MFCC）。根據人耳聽覺機理的研究發現，人耳對不同頻率的聲波有不同的聽覺敏感度。從200Hz到5000Hz的語音信號對語音的清晰度影響對大。

兩個響度不等的聲音作用於人耳時，則響度較高的頻率成分的存在會影響到對響度較低的頻率成分的感受，使其變得不易察覺，這種現象稱為掩蔽效應。由於頻率較低的聲音在內耳蝸基底膜上行波傳遞的距離大於頻率較高的聲音，故一般來說，低音容易掩蔽高音，而高音掩蔽低音較困難。在低頻處的聲音掩蔽的臨界帶寬較高頻要小。所以，人們從低頻到高頻這一段頻帶內按臨界帶寬的大小由密到疏安排一組帶通濾波器，對輸入信號進行濾波。將每個帶通濾波器輸出的信號能量作為信號的基本特徵，對此特徵經過進一步處理後就可以作為語音的輸入特徵。

由於這種特徵不依賴於信號的性質，對輸入信號不做任何的假設和限制，又利用了聽覺模型的研究成果。因此，這種參數比基於聲道模型的LPCC相比具有更好的Robustness，更符合人耳的聽覺特性，而且當信噪比降低時仍然具有較好的識別性能。

梅爾倒頻譜係數通常是用以下方法得到的：

- 將一訊號進行傅利葉轉換（Fourier transform）
- 將頻譜映射（mapping）至梅爾刻度，利用三角窗函數（triangular overlapping window）
- 取對數（logarithm）
- 取離散餘弦轉換（discrete cosine transform）
- MFCC是轉換後的頻譜


參考資料：
- http://mirlab.org/jang/books/audiosignalprocessing/speechFeatureMfcc_chinese.asp?title=12-2%20MFCC

### 資料下載
- https://drive.google.com/file/d/1QszsEG14awC5DwEWGIH0FGUobe7VtMnf/view?usp=sharing

### 安裝librosa
- pip install librosa

In [2]:
! pip install librosa

    100% |████████████████████████████████| 1.6MB 868kB/s ta 0:00:011
    100% |████████████████████████████████| 276kB 776kB/s ta 0:00:01
    100% |████████████████████████████████| 327kB 1.8MB/s ta 0:00:01
    100% |████████████████████████████████| 1.6MB 720kB/s ta 0:00:011
    100% |████████████████████████████████| 12.5MB 126kB/s ta 0:00:011
  Running setup.py bdist_wheel for librosa ... done
  Stored in directory: /Users/davidchiu/Library/Caches/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... done
  Stored in directory: /Users/davidchiu/Library/Caches/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... done
  Stored in directory: /Users/davidchiu/Library/Caches/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built librosa audioread resampy
You are using pip version 9.0.1, however version 18.1 is available.
Yo

In [4]:
import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm



In [5]:
# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [6]:
# Handy function to convert wav2mfcc
def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [7]:
# 從目錄夾讀取所有音檔，並將向量根據類別名稱存成 .npy 檔
def save_data_to_array(path=DATA_PATH, max_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)

### 準備訓練與測試資料集
- https://drive.google.com/file/d/1hThrCSQ5D8LNlVvkdqPbop1uTTsfl8lL/view?usp=sharing

In [33]:
def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)
    print(labels)
    # Getting first arrays
    X = np.load(DATA_PATH + labels[0] )
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(DATA_PATH + label )
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [34]:
def prepare_dataset(path=DATA_PATH):
    labels, _, _ = get_labels(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path  + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
            # Downsampling
            wave = wave[::3]
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data

In [35]:
def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]


In [36]:
%pwd

'/Users/davidchiu/course/tibamedl/CNN'

In [37]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

DATA_PATH = "./data/"
# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
# save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

['cat.npy', 'happy.npy', 'bed.npy']


In [48]:
X_train.shape

(3112, 20, 11, 1)

In [44]:
# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 10
batch_size = 100
num_classes = 3

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [49]:
model = Sequential()
# convolution
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))

# mat pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# flattening
model.add(Flatten())

# fully connected
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))


In [50]:
model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer='adam',
            metrics=['accuracy'])

In [51]:
model.fit(X_train, y_train_hot, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1, 
          validation_data=(X_test, y_test_hot))

Train on 3112 samples, validate on 2076 samples
Epoch 1/10
3112/3112 [==============================] - 2s 690us/step - loss: 1.7833 - acc: 0.4547 - val_loss: 0.7200 - val_acc: 0.7563
Epoch 2/10
3112/3112 [==============================] - 2s 512us/step - loss: 0.6923 - acc: 0.7105 - val_loss: 0.5162 - val_acc: 0.8068
Epoch 3/10
3112/3112 [==============================] - 2s 560us/step - loss: 0.5226 - acc: 0.7982 - val_loss: 0.4106 - val_acc: 0.8603
Epoch 4/10
3112/3112 [==============================] - 2s 539us/step - loss: 0.4251 - acc: 0.8454 - val_loss: 0.3326 - val_acc: 0.8844
Epoch 5/10
3112/3112 [==============================] - 2s 565us/step - loss: 0.3503 - acc: 0.8747 - val_loss: 0.3336 - val_acc: 0.8829
Epoch 6/10
3112/3112 [==============================] - 2s 581us/step - loss: 0.3297 - acc: 0.8850 - val_loss: 0.2785 - val_acc: 0.8979
Epoch 7/10
3112/3112 [==============================] - 2s 618us/step - loss: 0.2642 - acc: 0.9042 - val_loss: 0.2562 - val_acc: 0.9162


In [52]:
# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

In [53]:
predict('3bdf05d3_nohash_0.wav', model)

'happy.npy'

In [54]:
predict('7081436f_nohash_1.wav', model)

'bed.npy'

### Numpy 讀取與寫入

In [12]:
x = np.arange(10)
np.save('my_array', x)
np.load('my_array.npy')

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])